<a href="https://colab.research.google.com/github/hanbun-pom/siosio/blob/master/Yukapedia_%E3%83%91%E3%83%AF%E3%83%97%E3%83%AD%E7%94%A8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#君たちが打ったコメントを検索できます

不特定多数への公開は今の所ご遠慮下さい  
当ページ外で行われた操作についての責任は負いかねます  
@hanbun_pom
    
####🌸使い方🌸  
① 検索条件を入力  
② ↓にカーソルを合わせると表示される ▶ を押して実行

In [ ]:
import pandas as pd
import datetime
import requests

#@markdown ---
名前 = ""  #@param {type:"string"}
名前完全一致 = False  #@param {type:"boolean"} 
名前正規表現 = False  #@param {type:"boolean"} 
#@markdown ---
コメント = "^\u9EBB\u96C0\u30ED\u30DC\u30C3\u30C8$|^\u9EBB\u96C0\u30DE\u30B7\u30FC\u30F3$"  #@param {type:"string"}
コメント完全一致 = False  #@param {type:"boolean"}
コメント正規表現 = True  #@param {type:"boolean"} 
#@markdown ---
t秒前から再生 = 30 #@param {type:"slider", min:0, max:60, step:5}
#@markdown ---
配信日_以降 = '2019-12-24' #@param {type:"date"}
配信日_以前 = '2040-04-01' #@param {type:"date"}　


try:
    df2

except NameError:

    # 配信一覧　取得
    youtube = 'https://youtu.be/'
    master = 'https://raw.githubusercontent.com/hanbun-pom/siosio/master/'
    videos = pd.read_csv(master + 'videos_pawapuro.csv')

    # コメント一覧　作成
    df_list=[]
    for Id in videos['videoId']:
        csvURL = master + Id + '.tsv'
        res = requests.get(csvURL)
        if res.status_code == 200:
            tmp_df = pd.read_table(csvURL, header=None, names=['time','name','comment','videoId'],quotechar='"', quoting=3)
            tmp_df['videoId']= Id
            df_list.append(tmp_df)
        else:
            pass
    df = pd.concat(df_list, ignore_index=True)

    # 配信タイトル、公開時間
    df = df.merge(videos)
    df = df.rename(columns={'publishedAt': 'DateTime'})
    df['DateTime'] = pd.to_datetime(df['DateTime']).dt.date

    # 再生時間指定URL
    trans = str.maketrans({':': '',',': ''})
    s = df['time'].str.translate(trans).astype(int)
    s = s-40*(abs(s)//100)-2400*(abs(s)//10000)-t秒前から再生
    s = ((abs(s)+s)//2).astype(str)

    df['videoId'] = youtube + df['videoId'] + '?t=' + s + 's'

else:
    pass

#名前、コメント検索
def query_condition(data, col, query, is_perfect, is_regex):
    if is_perfect:
        return data[col].str.match(query) if is_regex else data[col] == query
    else:
        return data[col].str.contains(query, na=False, regex=is_regex)

name_condition = query_condition(df, "name", 名前, 名前完全一致, 名前正規表現)
comment_condition = query_condition(df, "comment", コメント, コメント完全一致, コメント正規表現)

df2 = df[name_condition & comment_condition].copy()

# 配信日指定
start = datetime.datetime.strptime(配信日_以降, '%Y-%m-%d').date()
end = datetime.datetime.strptime(配信日_以前, '%Y-%m-%d').date()
df3 = df2[df2['DateTime'] >= start]
df3 = df3[df3['DateTime'] <= end]

pd.set_option('display.max_rows', 10000)
df3


,time,name,comment,videoId,title,DateTime
24829,46:52,アンサイ/Uncy 安西実験工房,麻雀マシーン,https://youtu.be/l49nuiyQsPw?t=2782s,【NieR:Automata #1】ずっと気になってたゲームをやる！【Vtuber】,2020-05-01
82096,1:13:39,Association Sakura,麻雀マシーン,https://youtu.be/QJaCQm-oc6Q?t=4389s,【白鳥裁判】魚谷侑未プロと白鳥翔プロ呼んどいて、雑談するVtuberマジ？【🐟🌸🦢】,2020-04-29
111896,43:40,wildcat5314,麻雀マシーン,https://youtu.be/82NyKZTrAwI?t=2590s,【ASMR】バイノーラルマイクを買いました🎤【Vtuber】,2020-04-26
156586,7:40,アレアレア,麻雀マシーン,https://youtu.be/16ZnGTzrZ0k?t=430s,【ネコトモ】立派なネコ雀士にしてあげる【Vtuber】,2020-04-23
156602,7:45,白薔薇,麻雀マシーン,https://youtu.be/16ZnGTzrZ0k?t=435s,【ネコトモ】立派なネコ雀士にしてあげる【Vtuber】,2020-04-23
156916,9:12,としさん,麻雀マシーン,https://youtu.be/16ZnGTzrZ0k?t=522s,【ネコトモ】立派なネコ雀士にしてあげる【Vtuber】,2020-04-23
198305,48:05,いざよいさくや,麻雀ロボット,https://youtu.be/SRWpshDzqvc?t=2855s,天開司と楠栞桜の『てん x くす』三局目 出張版【楠栞桜視点】,2020-04-15
344687,58:55,いえろ〜ず,麻雀マシーン,https://youtu.be/NvMIkTa9SXs?t=3505s,【雀魂】コンビ打ち麻雀🔥本当の&quot;お引き&quot;見せます【楠栞桜視点】,2020-04-03
577849,2:34:15,CarMania385,麻雀ロボット,https://youtu.be/eBCbqMZajKs?t=9225s,【パズル探偵スカウト】楠栞桜、今日から探偵業始めます。【Vtuber】,2020-03-14
606458,49:21,カラスは可愛いのですよ?,麻雀マシーン,https://youtu.be/oBVYwlBFwa0?t=2931s,【雀魂】可愛い女の子と麻雀ができるって聞きました #1【Vtuber】,2020-03-06


2020/03/19 追加　完全一致検索  正規表現検索　動画URL生成  
2020/04/07 変更　GitHub版  